In [1]:
# https://github.com/Project-MONAI/tutorials/blob/main/3d_segmentation/torch/unet_training_array.py

import logging
import os
import sys
import tempfile
from glob import glob

import nibabel as nib
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:

import monai
from monai.data import ImageDataset, list_data_collate, create_test_image_3d, decollate_batch, DataLoader, Dataset, load_decathlon_datalist
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric, MeanIoU, HausdorffDistanceMetric
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    AsDiscrete,
    Compose,
    LoadImaged,
    RandCropByPosNegLabeld,
    RandRotate90d,
    ScaleIntensityd,
)
from monai.visualize import plot_2d_or_3d_image


monai.config.print_config()


In [2]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# Read the image and segmentation label

datalist = "/var/data/student_home/user/project_dir/monai_segmentation/monai_training/JSON_dir/train.json"
train_files = load_decathlon_datalist(data_list_file_path=datalist, is_segmentation=True, data_list_key='training')


In [3]:
X_train, X_val = train_test_split(train_files)
len(X_train), len(X_val)

(21, 8)

In [4]:
X_train[0]

{'fold': 0,
 'image': '/var/data/MONAI_Choroid_Plexus/dataset_monai_train_from_scratch/19_ChP.nii.gz',
 'label': '/var/data/MONAI_Choroid_Plexus/dataset_monai_train_from_scratch/labels/final/19_ChP.nii.gz'}

In [6]:
# define transforms for image and segmentation
train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        ScaleIntensityd(keys="image"),    
        RandCropByPosNegLabeld(
            keys=["image", "label"], label_key="label", spatial_size=[96, 96, 96], pos=1, neg=1, num_samples=4
        ),
        RandRotate90d(keys=["image", "label"], prob=0.5, spatial_axes=[0, 2]),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        ScaleIntensityd(keys="image"),
    ]
)

# define dataset, data loader

# create a training data loader
train_ds = monai.data.Dataset(data=X_train, transform=train_transforms)
# use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
train_loader = DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=list_data_collate,
    pin_memory=torch.cuda.is_available(),
)
# create a validation data loader
val_ds = monai.data.Dataset(data=X_val, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=4, collate_fn=list_data_collate)


In [7]:
# pick one image from DecathlonDataset to visualize and check the 4 channels
slice_nr = 230
val_data_example = val_ds[2]
print(f"image shape: {val_data_example['image'].shape}")
plt.figure("image", (24, 6))

plt.subplot(1, 2, 1)
plt.title(f"image channel ")
plt.imshow(val_data_example["image"][0, :, :, slice_nr].detach().cpu(), cmap="gray")
#plt.show()
# also visualize the 3 channels label corresponding to this image
print(f"label shape: {val_data_example['label'].shape}")
#plt.figure("label", (18, 6))

plt.subplot(1, 2, 2)
plt.imshow(val_data_example["label"][0, :, :, slice_nr].detach().cpu())
#plt.show()

image shape: torch.Size([1, 180, 240, 240])
label shape: torch.Size([1, 180, 240, 240])


In [9]:

# create UNet, DiceLoss and Adam optimizer
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
hauss_metric = HausdorffDistanceMetric(include_background=True, distance_metric="euclidean")
post_trans = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)]) 
# squashes its input values into a range between 0 and 1, and converts the data into a binary format. >0.5 = 1, <0.5 = 0

n_epochs = 1000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create UNet, DiceLoss and Adam optimizer
model = monai.networks.nets.UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(device)

# define loss function and optimizer
loss_function = monai.losses.DiceLoss(sigmoid=True)
loss_function2 = monai.losses.DiceCELoss(sigmoid=True)
loss_function3 = monai.losses.HausdorffDTLoss(sigmoid=True)

lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr)

# start a typical PyTorch training
val_interval = 2 # check the validation result every 2 epochs
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = list()
metric_values = list()
writer = SummaryWriter()
for epoch in range(n_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{n_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data["image"].to(device), batch_data["label"].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function3(outputs, labels) 
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        print(f"{step}/{epoch_len + 1}, train_loss: {loss.item():.4f}")
        writer.add_scalar("train_loss DiceCELoss", loss.item(), epoch_len * epoch + step) # add loss to tensorboard
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)   
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            val_images = None
            val_labels = None
            val_outputs = None
            for val_data in val_loader:
                val_images, val_labels = val_data["image"].to(device), val_data["label"].to(device)
                roi_size = (96, 96, 96)
                sw_batch_size = 4
                # divide img into smaller "windows" and apply model to each window in turn. Useful when the image is too large to
                # fit into memory all at once/ model has been trained on smaller patches of a larger image.
                val_outputs = sliding_window_inference(val_images, roi_size, sw_batch_size, model)
                # applying a post-processing transformation to the model's predictions and then decollating the batched output.
                val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]
                # compute metric for current iteration
                dice_metric(y_pred=val_outputs, y=val_labels)
            # aggregate the final mean dice result
            metric = dice_metric.aggregate().item()
            # reset the status for next validation round
            dice_metric.reset()

            metric_values.append(metric)
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), "best_metric_model_segmentation3d_dict.pth")
                print("saved new best metric model")
            print(
                "current epoch: {} current mean dice: {:.4f} best mean dice: {:.4f} at epoch {}".format(
                    epoch + 1, metric, best_metric, best_metric_epoch
                )
            )
            writer.add_scalar("val_mean_dice", metric, epoch + 1)
            # plot the last model output as GIF image in TensorBoard with the corresponding image and label
            plot_2d_or_3d_image(val_images, epoch + 1, writer, index=0, tag="image")
            plot_2d_or_3d_image(val_labels, epoch + 1, writer, index=0, tag="label")
            plot_2d_or_3d_image(val_outputs, epoch + 1, writer, index=0, tag="prediction")

print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
writer.close()



    

----------
epoch 1/1000


single channel prediction, `include_background=False` ignored.


1/11, train_loss: 421.0492
2/11, train_loss: 527.2701
3/11, train_loss: 286.7168
4/11, train_loss: 402.7094
5/11, train_loss: 176.9380
6/11, train_loss: 578.3251
7/11, train_loss: 510.9839
8/11, train_loss: 194.1223
9/11, train_loss: 307.5004
10/11, train_loss: 375.4178
11/11, train_loss: 140.6078
epoch 1 average loss: 356.5128
----------
epoch 2/1000
1/11, train_loss: 408.3312
2/11, train_loss: 172.6046
3/11, train_loss: 255.5466
4/11, train_loss: 234.3442
5/11, train_loss: 341.7447
6/11, train_loss: 374.4511
7/11, train_loss: 303.7993
8/11, train_loss: 370.5381
9/11, train_loss: 160.1056
10/11, train_loss: 225.2174
11/11, train_loss: 342.5543
epoch 2 average loss: 289.9307
saved new best metric model
current epoch: 2 current mean dice: 0.0016 best mean dice: 0.0016 at epoch 2
----------
epoch 3/1000
1/11, train_loss: 180.3946
2/11, train_loss: 171.3702
3/11, train_loss: 572.0248
4/11, train_loss: 415.0325
5/11, train_loss: 557.4181
6/11, train_loss: 236.8447
7/11, train_loss: 100.639

In [13]:
model.load_state_dict(torch.load(os.path.join(root_dir, "best_metric_model.pth")))
model.eval()
with torch.no_grad():
    # select one image to evaluate and visualize the model output
    val_input = val_ds[6]["image"].unsqueeze(0).to(device)
    roi_size = (128, 128, 64)
    sw_batch_size = 4
    val_output = inference(val_input)
    val_output = post_trans(val_output[0])
    plt.figure("image", (24, 6))
    for i in range(4):
        plt.subplot(1, 4, i + 1)
        plt.title(f"image channel {i}")
        plt.imshow(val_ds[6]["image"][i, :, :, 70].detach().cpu(), cmap="gray")
    plt.show()
    # visualize the 3 channels label corresponding to this image
    plt.figure("label", (18, 6))
    for i in range(3):
        plt.subplot(1, 3, i + 1)
        plt.title(f"label channel {i}")
        plt.imshow(val_ds[6]["label"][i, :, :, 70].detach().cpu())
    plt.show()
    # visualize the 3 channels model output corresponding to this image
    plt.figure("output", (18, 6))
    for i in range(3):
        plt.subplot(1, 3, i + 1)
        plt.title(f"output channel {i}")
        plt.imshow(val_output[i, :, :, 70].detach().cpu())
    plt.show()


NameError: name 'model' is not defined